<a href="https://colab.research.google.com/github/Nirika-Lamichhane/Minor_Project-5-24-25-36-/blob/main/language_identification_to_be_later_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pandas as pd
from sklearn.utils import shuffle

# Load your dataset
df = pd.read_csv("/content/lang_identification_1.csv")

# Shuffle the dataset
df = shuffle(df, random_state=42).reset_index(drop=True)

# Save the shuffled dataset (optional)
df.to_csv("shuffled_dataset.csv", index=False)

# Quick check
print(df.head())
print(df['label'].value_counts())


                                                text       label
0                                Thumbnail is lit 😂😂          EN
1                  माटोलाई हाल्ने हो जितोस या हारोस्      NE_DEV
2          nicholous❌ चाँडै (निको होस) mental cramps  CODE_MIXED
3                                      He is right ❤          EN
4  Twist ta Aayo ta kulman neskeya aaba raja aaun...      NE_ROM
label
EN            496
NE_ROM        385
NE_DEV        275
CODE_MIXED    214
Name: count, dtype: int64


In [45]:
# Step 1: Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Step 2: Load your shuffled dataset
# Assuming your dataset is CSV with two columns: 'text' and 'label'
df = pd.read_csv('/content/lang_identification_1.csv')  # replace with your CSV path

texts = df['text'].astype(str).tolist()
labels = df['label'].tolist()

# Step 3: Split into training and testing sets (80%-20%)
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

# Step 4: TF-IDF vectorization
tfidf_vectorizer = TfidfVectorizer(
    analyzer='char_wb',   # character n-grams help for short Roman and noisy text
    ngram_range=(2, 5),   # 2-5 char ngrams
    lowercase=True,
    max_features=10000    # limit features to keep it manageable
)

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Step 5: Train SVM
svm_clf = SVC(
    kernel='linear',   # linear kernel works well for text
    C=1.0,             # regularization parameter
    probability=True,  # allows probability output if needed
    class_weight='balanced',  # handle class imbalance
    random_state=42
)

svm_clf.fit(X_train_tfidf, y_train)

# Step 6: Evaluate
y_pred = svm_clf.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.8832116788321168

Classification Report:
               precision    recall  f1-score   support

  CODE_MIXED       0.76      0.58      0.66        43
          EN       0.87      0.97      0.92        99
      NE_DEV       0.96      1.00      0.98        55
      NE_ROM       0.89      0.86      0.87        77

    accuracy                           0.88       274
   macro avg       0.87      0.85      0.86       274
weighted avg       0.88      0.88      0.88       274



In [14]:
!pip uninstall -y numpy
!pip install numpy==1.26.4



Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 86.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.36.3 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.

In [46]:
import fasttext
import pandas as pd
from sklearn.model_selection import train_test_split

# 1. Load your dataset (CSV with columns: text, label)
df = pd.read_csv("/content/lang_identification_1.csv")

# 2. Convert to FastText format
# FastText expects each line as: __label__<label> <text>
df['fasttext_format'] = "__label__" + df['label'] + " " + df['text']

# 3. Split into training and validation sets
train_texts, val_texts = train_test_split(df['fasttext_format'], test_size=0.2, random_state=42, shuffle=True)

# 4. Save the datasets to temporary text files for FastText
with open("train.txt", "w", encoding="utf-8") as f:
    for line in train_texts:
        f.write(line + "\n")

with open("valid.txt", "w", encoding="utf-8") as f:
    for line in val_texts:
        f.write(line + "\n")

# 5. Train the supervised FastText model
model = fasttext.train_supervised(
    input="train.txt",
    lr=0.5,              # learning rate, higher value for faster convergence
    epoch=50,            # number of training iterations over the dataset
    wordNgrams=2,        # use bigrams for capturing context (helps code-mixed)
    dim=300,             # size of word vectors
    minn=2, maxn=5,      # subword information for OOV words and Roman Nepali
    bucket=200000,       # hash bucket size for subwords
    loss='softmax',      # softmax is standard for multi-class classification
    thread=4,            # number of CPU threads to use
)

# 6. Evaluate on validation set
result = model.test("valid.txt")
print("FastText Validation Metrics:")
print(f"Samples: {result[0]}, Precision: {result[1]:.4f}, Recall: {result[2]:.4f}")

# 7. Save the trained model for later use
model.save_model("fasttext_language_model.bin")




FastText Validation Metrics:
Samples: 274, Precision: 0.8905, Recall: 0.8905


In [47]:
import fasttext
import re
import numpy as np

# Load FastText model
ft_model = fasttext.load_model("fasttext_language_model.bin")

# --- Helper: text cleaning ---
def clean_text(text):
    text = str(text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# --- Word count ---
def word_count(text):
    return len(text.split())

# --- SVM prediction ---
def predict_svm(text):
    text_tfidf = tfidf_vectorizer.transform([text])
    pred = svm_clf.predict(text_tfidf)[0]
    prob = max(svm_clf.predict_proba(text_tfidf)[0])
    return pred, prob

# --- FastText prediction ---
def predict_fasttext(text):
    labels, probs = ft_model.predict(text)

    # Force safe conversion (NumPy 2.x compatible)
    labels = list(labels)
    probs = np.asarray(probs)

    label = labels[0].replace("__label__", "")
    confidence = float(probs[0])

    return label, confidence

# --- Hybrid prediction ---

CONFIDENCE_THRESHOLD = 0.57
def hybrid_language_detect(text, threshold=6):
    text = clean_text(text)
    wc = word_count(text)

    if wc <= threshold:
        label, confidence = predict_svm(text)
        source = "SVM"
    else:
        label, confidence = predict_fasttext(text)
        source = "FastText"

    if confidence < CONFIDENCE_THRESHOLD or wc < 2:
        return {
            "text": text,
            "predicted_label": "UNKNOWN",
            "confidence": round(float(confidence), 4),
            "model_used": source,
            "word_count": wc
        }

    return {
        "text": text,
        "predicted_label": label,
        "confidence": round(confidence, 4),
        "model_used": source,
        "word_count": wc
    }


In [50]:
examples = [
    "kp chor 👍",
    "सबै व्यक्तिहरू जन्मजात स्वतन्त्र हुन्",
    "lau yaar not true",
    "vote for balen now or never 🇳🇵",
    "Jhapa ma aunu parcha",
    "xya"
]

for e in examples:
    print(hybrid_language_detect(e))


{'text': 'kp chor 👍', 'predicted_label': 'EN', 'confidence': 0.6531, 'model_used': 'SVM', 'word_count': 3}
{'text': 'सबै व्यक्तिहरू जन्मजात स्वतन्त्र हुन्', 'predicted_label': 'NE_DEV', 'confidence': 0.9779, 'model_used': 'SVM', 'word_count': 5}
{'text': 'lau yaar not true', 'predicted_label': 'CODE_MIXED', 'confidence': 0.7713, 'model_used': 'SVM', 'word_count': 4}
{'text': 'vote for balen now or never 🇳🇵', 'predicted_label': 'EN', 'confidence': 0.9994, 'model_used': 'FastText', 'word_count': 7}
{'text': 'Jhapa ma aunu parcha', 'predicted_label': 'NE_ROM', 'confidence': 0.891, 'model_used': 'SVM', 'word_count': 4}
{'text': 'xya', 'predicted_label': 'UNKNOWN', 'confidence': 0.736, 'model_used': 'SVM', 'word_count': 1}
